In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import prince

In [2]:
df = pd.read_csv('data/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2018.csv')
df.head()

C:\Users\Urvi\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Emergency Department Indicator,Total Charges,Total Costs
0,Capital/Adirond,Columbia,1001000.0,146.0,Columbia Memorial Hospital,50 to 69,125,F,White,Not Span/Hispanic,...,Major,Major,Medical,"Managed Care, Unspecified",Self-Pay,NaN,NaN,True,17591.41,3570.06
1,New York City,Manhattan,7002009.0,1445.0,Harlem Hospital Center,50 to 69,100,F,Black/African American,Not Span/Hispanic,...,Extreme,Extreme,Surgical,Medicare,Medicaid,NaN,NaN,True,106190.21,78913.24
2,New York City,Manhattan,7002009.0,1445.0,Harlem Hospital Center,50 to 69,100,F,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicaid,NaN,NaN,NaN,False,5600.00,4161.53
3,New York City,Kings,7001016.0,1301.0,Kings County Hospital Center,18 to 29,112,M,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicaid,Medicaid,NaN,NaN,True,15965.54,11341.30
4,New York City,Manhattan,7002054.0,1464.0,New York-Presbyterian Hospital - Columbia Pres...,0 to 17,104,M,Multi-racial,Not Span/Hispanic,...,Major,Moderate,Medical,Medicaid,Medicaid,Self-Pay,NaN,True,36846.83,10240.02


In [3]:
# Cleaning steps
# Drop any columns with > 30% null values
df_cleaned = df.drop(['Birth Weight', 'Payment Typology 3', 'Payment Typology 2', 'CCSR Procedure Code', 'CCSR Procedure Description'], axis=1)
df_cleaned.dropna(inplace=True)
df_cleaned = df_cleaned[df_cleaned['Length of Stay'] != '120 +']
df_cleaned_sample = df_cleaned.sample(100000)

In [4]:
y = df_cleaned_sample[['Length of Stay']]
X = df_cleaned_sample.loc[:, ~df_cleaned_sample.columns.isin(['Length of Stay', 'Total Costs', 'Total Charges'])]
print(y.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

for column in X_train.columns:
    if X_train[column].dtype.name == 'bool':
        X_train[column] = X_train[column].astype('object')

for column in X_train.columns:
    print(column, X_train[column].dtype)

Index(['Length of Stay'], dtype='object')
Hospital Service Area object
Hospital County object
Operating Certificate Number float64
Permanent Facility Id float64
Facility Name object
Age Group object
Zip Code - 3 digits object
Gender object
Race object
Ethnicity object
Type of Admission object
Patient Disposition object
Discharge Year int64
CCSR Diagnosis Code object
CCSR Diagnosis Description object
APR DRG Code float64
APR DRG Description object
APR MDC Code float64
APR MDC Description object
APR Severity of Illness Code float64
APR Severity of Illness Description object
APR Risk of Mortality object
APR Medical Surgical Description object
Payment Typology 1 object
Emergency Department Indicator object


In [5]:
print(X_train['Emergency Department Indicator'].head())

2101767     True
1957624    False
1436279    False
1758592     True
409968      True
Name: Emergency Department Indicator, dtype: object


In [6]:
'''famd = prince.FAMD(
    n_components=10, 
    n_iter=3,
    random_state=42)
famd.fit_transform(X_train)
print(famd.explained_inertia_.sum())'''

'famd = prince.FAMD(\n    n_components=10, \n    n_iter=3,\n    random_state=42)\nfamd.fit_transform(X_train)\nprint(famd.explained_inertia_.sum())'

In [10]:
optimal_components = pd.DataFrame(columns = ['num_components', 'explained_variance'])

for i in range(1, 50):
    print(i)
    # Get val of x
    num_components = i
    
    # Initialize FAMD
    famd = prince.FAMD(n_components = i, n_iter = 3, random_state = 42)
    famd.fit_transform(X_train)
    
    # Calculate explained variance
    explained_variance = famd.explained_inertia_.sum()
    
    # Insert into dataframe
    row = {'num_components': num_components, 'explained_variance' : explained_variance}
    optimal_components = optimal_components.append(row, ignore_index = True)
    


# Print max 
optimal_components.loc[optimal_components['explained_variance'] == optimal_components['explained_variance'].max()]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


,num_components,explained_variance
48,49.0,0.122538


In [15]:
X_train = X_train[['Age Group', 'Type of Admission', 'Patient Disposition', 'APR Severity of Illness Code', 'APR Severity of Illness Description', 'APR Risk of Mortality', 'Emergency Department Indicator']]

X_train['APR Severity of Illness Code'] = X_train['APR Severity of Illness Code'].astype(str)

In [34]:
mca = prince.MCA(
     n_components=2,
     n_iter=3,
     copy=True,
     check_input=True,
     engine='auto',
     random_state=42
 )
mca = mca.fit(X_train)
print(mca.total_inertia_)
print(mca.explained_inertia_)

X_train_transformed = mca.transform(X_train)


5.428571428571429
[0.10076833972655405, 0.07070705907558329]


In [35]:
regr = LinearRegression()
  
regr.fit(X_train_transformed, y_train)

X_test = X_test[['Age Group', 'Type of Admission', 'Patient Disposition', 'APR Severity of Illness Code', 'APR Severity of Illness Description', 'APR Risk of Mortality', 'Emergency Department Indicator']]

X_test['APR Severity of Illness Code'] = X_test['APR Severity of Illness Code'].astype(str)
X_test['Emergency Department Indicator'] = X_test['Emergency Department Indicator'].astype(str)

print(X_test.dtypes)

X_test_transformed = mca.fit_transform(X_test)
preds = regr.predict(X_test_transformed)
mae = metrics.mean_absolute_error(y_test, preds)
mse = metrics.mean_squared_error(y_test, preds)
r2 = metrics.r2_score(y_test, preds)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))

Age Group                              object
Type of Admission                      object
Patient Disposition                    object
APR Severity of Illness Code           object
APR Severity of Illness Description    object
APR Risk of Mortality                  object
Emergency Department Indicator         object
dtype: object
The model performance for testing set
--------------------------------------
MAE is 3.4596929488162296
MSE is 45.72567480958954
R2 score is 0.12240706167430215


In [36]:
from sklearn.linear_model import Ridge

clf = Ridge(alpha=1.0)
clf.fit(X_train_transformed, y_train)

preds = clf.predict(X_test_transformed)
mae = metrics.mean_absolute_error(y_test, preds)
mse = metrics.mean_squared_error(y_test, preds)
r2 = metrics.r2_score(y_test, preds)

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))

The model performance for testing set
--------------------------------------
MAE is 3.4596961280203025
MSE is 45.7256734013026
R2 score is 0.12240708870293926
